In [5]:
#filedrive="/content/drive/My Drive/0Satellite_Images_Unsupervised/MNIST_PyTorch"
#from google.colab import drive
#drive.mount('/content/drive')
filedrive="data/0Satellite_Images_Unsupervised/MNIST_PyTorch"

import torch, torchvision
from torch import nn, optim

In [3]:
class Classifier(nn.Module):
    def __init__(self, bands_nb, patch_size):

        input_size = (bands_nb, patch_size, patch_size)
      
        super().__init__()
        self.layer1 = nn.Linear(784, 20)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(20, 10)
        self.layer3 = nn.Linear(784*32, 10)

        self.conv11 = nn.Conv2d(bands_nb, 32, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(32)
        self.conv12 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(32)
        self.conv13 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv14 = nn.Conv2d(32, 1, kernel_size=3, padding=1)

        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()
#        self.pool = nn.MaxPool2d(2, 2)
#        self.l2norm = L2()

    def forward(self, x):
        x=F.relu(self.conv11(x)) #        x = self.pool(nn.functional.relu(self.conv1(x)))
        x=F.relu(self.conv12(x))
         # flatten all dimensions except batch
#        x = F.relu(self.layer3(x))
        x=F.relu(self.conv13(x))
        return self.conv14(x)


In [6]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(
    root=filedrive, train=False, transform=transform, download=True)

100.0%


Extracting data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw/train-images-idx3-ubyte.gz to data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw



102.8%


Extracting data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw/train-labels-idx1-ubyte.gz to data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw



100.0%


Extracting data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw/t10k-images-idx3-ubyte.gz to data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw



112.7%

Extracting data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/0Satellite_Images_Unsupervised/MNIST_PyTorch/MNIST/raw

Processing...
Done!



/opt/miniconda/envs/mlapp_py3.6/lib/python3.6/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [8]:
epochs = 5
for epoch in range(epochs):
    loss = 0
    for batch_features, batch_labels in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)

        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()

        # compute reconstructions
        outputs = model(batch_features)
        #outputs = outputs.view(-1).to(device)

        # compute training reconstruction loss

        train_loss = criterion(outputs, batch_labels.to(device))

        # compute accumulated gradients
        train_loss.backward()

        # perform parameter update based on current gradients
        optimizer.step()

        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()

    # compute the epoch training loss
    loss = loss / len(train_loader)

    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

NameError: name 'device' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
total = 0
correct = 0
print(test_loader.dataset)
for batch_features, labels in test_loader:
    batch_features = batch_features.view(-1, 784).to(device)  
    outputs = model(batch_features)
    #print(outputs.shape)

    #print('IMAGE')

    #plt.imshow(batch_features[0].cpu().detach().view(28, 28).numpy())
    #plt.show()     

    #print(np.argmax(outputs[0].cpu().detach().numpy()))   

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))